<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/%ED%95%9C%EA%B8%80%EB%AC%B8%EC%9E%A5%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import nltk
nltk.download('punkt')

urls = ['https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-ABC%20%EC%82%B4%EC%9D%B8%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EA%B7%B8%EB%A6%AC%EA%B3%A0%20%EC%95%84%EB%AC%B4%EB%8F%84%20%EC%97%86%EC%97%88%EB%8B%A4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%82%98%EC%9D%BC%EA%B0%95%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A7%8C%EC%B0%AC%ED%9A%8C%EC%9D%98%2013%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%A9%94%EC%86%8C%ED%8F%AC%ED%83%80%EB%AF%B8%EC%95%84%EC%9D%98%20%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%AA%A9%EC%82%AC%EA%B4%80%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B2%99%EC%96%B4%EB%A6%AC%20%EB%AA%A9%EA%B2%A9%EC%9E%90.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EB%B9%84%EB%B0%80%20%EC%84%9C%EB%A5%98%EB%A5%BC%20%EB%85%B8%EB%A0%A4%EB%9D%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%8A%A4%ED%8E%98%EC%9D%B8%EA%B6%A4%EC%A7%9D%EC%9D%98%20%EB%B9%84%EB%B0%80.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%84%ED%8C%8C%ED%8A%B8%EC%97%90%20%EB%82%98%ED%83%80%EB%82%9C%20%EC%9A%94%EC%A0%95.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%95%A0%ED%81%AC%EB%A1%9C%EC%9D%B4%EB%93%9C%20%EC%82%B4%EC%9D%B8%20%EC%82%AC%EA%B1%B4.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%98%88%EA%B3%A0%20%EC%82%B4%EC%9D%B8.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%A5%90%EB%8D%AB.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%EC%BB%A4%ED%8A%BC.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%81%AC%EB%A6%AC%EC%8A%A4%EB%A7%88%EC%8A%A4%20%ED%91%B8%EB%94%A9%EC%9D%98%20%EB%AA%A8%ED%97%98.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%91%B8%EB%A5%B8%EC%97%B4%EC%B0%A8%EC%9D%98%EC%A3%BD%EC%9D%8C.txt',
        'https://raw.githubusercontent.com/dolmani38/Summary/master/data/%EC%95%A0%EA%B1%B0%EC%84%9C%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%8B%B0-%ED%99%94%EC%9A%94%EC%9D%BC%20%ED%81%B4%EB%9F%BD%EC%9D%98%20%EC%82%B4%EC%9D%B8.txt']

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_korean_sentences(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40:
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
korean_sentences = collect_korean_sentences(urls)

In [ ]:
len(korean_sentences)

18036

In [ ]:
korean_sentences[100]

'그러나 당신이 받으셨다는 그 이상한 편지도 고려해 보고 싶습니다, 포아로 씨.'

# 한글 뉴스 크롤러

In [2]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if False:
    f = open("/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv", "a")
    f.write("title,contents\r\n")
    f.close() 

In [3]:
import sys
import requests
import pandas as pd
import datetime
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool

client_id = "waXxTjNdGjzj8xtT3nbF" #1.에서 취득한 아이디 넣기
client_secret = "H6mejnkuYx"  #1. 에서 취득한 키 넣기


# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    txt = txt.replace('”','')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def clean_html(x):
    x = re.sub("\&\w*\;","",x)
    x = re.sub("<.*?>","",x)
    return x

def get_news_urls(search_word = '코로나',max_display = 100):
    encode_type = 'json' #출력 방식 json 또는 xml
    #max_display = 100 #출력 뉴스 수
    sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
    start = 1 # 출력 위치

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #print(url)
    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    df = None
    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    if r.status_code == 200:
        #요청 결과 보기 200 이면 정상적으로 요청 완료
        df = pd.DataFrame(r.json()['items'])
        df['title'] = df['title'].apply(lambda x: clean_html(x))
        df['description'] = df['description'].apply(lambda x: clean_html(x))
    return df

def get_news_title_contents(url):
    headers = {'User-Agent':'Chrome/66.0.3359.181'}
    main_news = []
    result = requests.get(url,headers=headers)
    if result.status_code == 200:
        bs_obj = BeautifulSoup(result.content, "html.parser")
        #print(bs_obj)
        try:
            title = bs_obj.find("h3",{"id":"articleTitle"}).text
            body = bs_obj.find("div",{"class":"_article_body_contents"}).get_text("\n").replace("// flash 오류를 우회하기 위한 함수 추가","").replace("function _flash_removeCallback() {}","")
            #aid = bs_obj.find("div",{"class":"article_header"}).find("img")["title"]
            main_news = [clean_text(title),clean_text(body)]
        except:
            pass
            #print("Unexpected error:", sys.exc_info()[0])
    return main_news    

In [4]:
def make_corpus(search_word = '코로나'):
    f = open("/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv", "a")
    df = get_news_urls(search_word = search_word,max_display = 100)
    if df is not None:
        for url in df['link']:
            if url.startswith('https://news.naver.com/'):
                #print(url)
                news = get_news_title_contents(url)
                if (len(news) == 2):
                    print('\rtitle:',news[0],end="", flush=True)
                    f.write(news[0] + "," + news[1] + "\r\n")
    f.close() 

In [5]:
 if False:
    keywords = ['뉴스','속보','스포츠','광명','금융','자동','인터뷰','대출','신도시','바이오','경쟁','공무원','재난','신도시','청약','주문','협력','교회','충성','요양','병원','백신']
    keywords += ['택시','유족','방치','DNA','중대본','공수처','검찰','서울','시장','단일화','시간표','야권','협상','여론','중도','특검','포비아','구토','수도권','증강현실','게임','선물','시계','장사','거리','허탈','IPO','상견례','결정','악재','외국계','희소식','배터리','질주']
    keywords += ['미세먼지','봄비','전국','소식','강원','우산','미나리','뉴욕','미국','출연','배우','아카네미','기생충','오스카','방탄','소년','BTS','앨범','판매','누리꾼','클릭','차트','음반','폭스바겐','아틀라스','유지','비용','세단','화사','궁궐','왕릉','유적','개화','절정','4월','문화재','조선','헤드','라인','국가','대표']
    keywords += ['접종','중단']
    keywords += ['유럽','국가','예방','선업','특정','태국','연기','혈전','관계','확인','차원','혈액','응고','총리','접종','취소','부작용','갈등','발화','홍콩','보고','입법부','인사','자리','선거','개편','배척','통과','투표','버튼','미소','시진핑']
    for word in keywords:
        print('keyword:',word)
        make_corpus(search_word = word)

In [8]:
import random
source = "전북서 본부장으로 일할 때 바람직하지 않은 일 했다 3월3일 오후 경남 진주시 충무공동 한국토지주택공사(LH) 본사 앞에 빨간 신호등이 켜 있다. 문재인 대통령은 이날 LH 직원들의 100억원대 광명·시흥 신도시 투기 의혹에 대해 철저한 조사를 지시했다. ⓒ 연합뉴스 직원들의 광명·시흥 3기 신도시 지역 투기 의혹에 휩싸인 한국토지주택공사(LH)의 고위 간부가 12일 스스로 목숨을 끊었다. 경기남부경찰청 등에 따르면 이날 오전 9시40분께 경기도 성남시 분당의 한 아파트 화단에서 LH 현직 고위 간부 A씨가 쓰러진 채로 발견됐다. A씨는 병원으로 이송됐지만 사망했다.   A씨는 전북에서 본부장으로 근무할 때 바람직하지 않은 일을 했다. 괴롭다. 국민에게 죄송하다는 내용의 유서를 남긴 것으로 전해졌다. 경찰은 아파트 주변 폐쇄(CCTV)회로 등을 조사한 결과 현재까지 범죄 혐의점은 발견되지 않았으며 사망 원인 등에 대한 추가 조사를 벌이고 있다고 밝혔다."

source = source.replace(" ","")
source = source.replace(".","")
for i in range(1000):
    keyword = random.choice(source)+random.choice(source)
    try:
        make_corpus(search_word = keyword)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])        

title: 한-몽골 국장급 환경정책협의체 만든다Unexpected error: <class 'KeyError'>
title: 올해 13조 번다 D램 호황 앞서본 최태원의 인텔 인수Unexpected error: <class 'KeyError'>
title: 韓. 러공단 개발 관련 정부간 협정체결 추진Unexpected error: <class 'KeyError'>
title: 원주이씨 단양종중 소장 유물 31점 충북문화재 지정Unexpected error: <class 'KeyError'>
title: 30년이 지난 지금 봐도 놀랍다. 서울시립미술관 이불-시작Unexpected error: <class 'KeyError'>
title: 쾰른 성범죄 악몽 재현 없다…독일 새해 행사 대규모 경찰 병력 투입Unexpected error: <class 'KeyError'>
title: 日 백신 본격 접종은 4월 말에나? 일정 차질 이유는…Unexpected error: <class 'KeyError'>
title: ‘슬기로운 의사생활’ 안은진 위급한 산모 수술…김대명 위로에 ‘눈물’Unexpected error: <class 'KeyError'>
title: 장모와 딸에게 꼬리 잡힌 바람男들Unexpected error: <class 'KeyError'>
title: <초점>시. 군 공무원 5급 승진제도 개선돼야Unexpected error: <class 'KeyError'>
Unexpected error: <class 'KeyError'>
title: 안양도시공사 4일간 공영주차장 무료개방Unexpected error: <class 'KeyError'>
title: 사천시 농촌 지역 마을 리더 역량 강화 현장견학 시행Unexpected error: <class 'KeyError'>
Unexpected error: <class 'KeyError'>
title: 자녀 성교육 어떻게 해야 할지 막막하다면Unexpected error: <class 'KeyError'>


KeyboardInterrupt: ignored

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
37227,이런 광경 얼마 만인지…쏟아진 中 관광객에 웃은 면세점,한한령 이후 최대 규모 5000명 인센티브 관광 중국인 관광객 증가 터닝포인트 되길...
37228,文 투기와의 전쟁에서 지지 않을 것… 고강도 규제 예고편?,부동산 시장 안정 실수요자 보호 투기 억제에 대한 정부의 의지는 확고 풍선효과 계속...
37229,그룹 신화 멤버 김동완씨 덕분에 만나게 된 외국인 독자,[내가 만난 독자] 소년의 레시피를 읽고 쓰고 일본어로 옮기는 기쿠치 미유키씨 [오...
37230,[여의도 사사건건] 내일부터 21대 총선 예비후보 등록…선거법 어디로?,- 낙후된 지역 SOC 신규 사업 예산 확보는 잘한 일…그래야 균형 발전 - 4+1...


In [7]:
df['contents'][100]

'전북서 본부장으로 일할 때 바람직하지 않은 일 했다 3월3일 오후 경남 진주시 충무공동 한국토지주택공사(LH) 본사 앞에 빨간 신호등이 켜 있다. 문재인 대통령은 이날 LH 직원들의 100억원대 광명·시흥 신도시 투기 의혹에 대해 철저한 조사를 지시했다. ⓒ 연합뉴스 직원들의 광명·시흥 3기 신도시 지역 투기 의혹에 휩싸인 한국토지주택공사(LH)의\xa0고위 간부가 12일 스스로 목숨을 끊었다. 경기남부경찰청 등에 따르면 이날 오전 9시40분께 경기도 성남시 분당의 한 아파트 화단에서 LH 현직 고위 간부 A씨가 쓰러진 채로 발견됐다. A씨는 병원으로 이송됐지만 사망했다. \xa0 A씨는 전북에서 본부장으로 근무할 때 바람직하지 않은 일을 했다. 괴롭다. 국민에게 죄송하다는 내용의 유서를 남긴 것으로 전해졌다. 경찰은\xa0아파트 주변 폐쇄(CCTV)회로 등을 조사한 결과 현재까지 범죄 혐의점은 발견되지 않았으며 사망 원인 등에 대한 추가 조사를 벌이고 있다고 밝혔다. \xa0 ※ 우울감 등 말하기 어려운 고민이 있거나 주변에 이런 어려움을 겪는 가족·지인이 있으면 자살 예방 핫라인 ☎1577-0199 희망의 전화 ☎129 생명의 전화 ☎1588-9191 청소년 전화 ☎1388 등에서 24시간 전문가의 상담을 받을 수 있습니다. 이혜영 기자 zero@sisajournal. com <저작권자 ⓒ 시사저널 무단 전재 및 재배포 금지> ☞ 네이버에서 시사저널 뉴스를 받아 보세요 ▶ 시사저널 최신호 보기'

In [10]:
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))


In [11]:
sentences = []

for raw_text in df['contents']:
    try:
        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if len(txt) > 40:
                    #ok = True
                    #for f in filters:
                    #    if len(f.findall(txt)) > 0:
                    #        ok = False
                    #        print('fitered:',txt)
                    #if ok:
                    if any(chr.isdigit() for chr in txt):
                        pass
                    else:
                        sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])                         

Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>


In [12]:
len(sentences)

407555

In [13]:
sentences[254157]

'축산물 공판장에서 무더기로 확진자가 나온 데다 어린이집 병원 목욕탕·사우나 제조업장 등에서도 신규 감염이 잇따르고 있습니다.'

In [14]:
df1 = pd.DataFrame({'sentence':sentences})

In [15]:
df1

,sentence
0,연합뉴스 문재인 대통령이 변창흠 국토교통부 장관의 사의 표명을 사실상 수용했다.
1,문 대통령은 변 장관의 사의 표명에 책임지는 모습을 보일 수밖에 없다고 밝혔다.
2,이에 앞서 변창흠 국토교통부 장관은 LH 땅 투기 의혹 사건과 관련한 책임론에 대해...
3,변 장관은 이날 국회 국토교통위원회 전체회의에 참석해 “LH 사태로 국민들이 걱정하...
4,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...
407550,마무리를 해야 하는데 만약에 선거법 올린다고 해도 필리버스터가 있기 때문에 쉽지 않...
407551,의사 진행권은 의장한테 있기 때문에 회기 정하는 것은 필리버스터에 적용 안 된다고 ...
407552,JC 중앙회장도 하고 굉장히 똑똑한 친구인데 아버지 뒤를 이어서 자기가 아버지가 공...
407553,그리고 의정부에서 출마한다는데 의정부 시민들도 문희상 의장의 아들이 출마하는지 안 ...


In [16]:
df1.to_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_sentences.csv')